# Mask Refine Demo

## Set Up & Configuration

First, we'll import the generally required packages:

In [1]:
import imgaug.augmenters as iaa
import numpy as np
from os.path import exists

from mask_refine.mask_refine import MaskRefineSubnet
from mask_refine.mask_refine import MaskRefineModule
from opt_flow.opt_flow import TensorFlowPWCNet
from train.davis2017_dataset import *
from train.datautils import splitd
from train.viz import *

C:\Program Files\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


Now, we'll need to define some useful constants and parameters. (Change these to the appropriate numbers for your system and application.)

We're also creating the dataset and the generator here.

In [ ]:
DATASET_PATH = '../data/DAVIS/'
OPTICAL_FLOW_WEIGHTS = './opt_flow/models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000'
MASK_REFINE_WEIGHTS = './logs/davis_unet_weights.h5'

assert exists(DATASET_PATH), 'Can\'t find path for dataset'
# we don't check the optical flow one, since that is only a 'virtual path' of sorts
assert exists(MASK_REFINE_WEIGHTS), 'Can\'t find path for mask refine model'

AUGMENTATION_SEQUENCE = iaa.Sequential([
    iaa.Multiply((0.25, 0.95)),
    iaa.ElasticTransformation(alpha=(2000, 10000), sigma=(20, 100)),
    iaa.GaussianBlur(sigma=(0, 20)),
    iaa.Sometimes(0.25, iaa.Multiply((0.5, 0.75))),
    iaa.MultiplyElementwise((0.8, 1.1)),
    iaa.Sometimes(0.1, iaa.GaussianBlur(sigma=(5, 100))),
    iaa.AdditiveGaussianNoise(scale=(1, 25))
])

DATASET = get_trainval(DATASET_PATH)
IMAGE_SHAPE = DATASET.size
GENERATOR = DATASET.paired_generator(AUGMENTATION_SEQUENCE)

Now, we create the optical flow model, the mask refine subnet, and the overall mask refine module.

In [ ]:
pwc_net = TensorFlowPWCNet(IMAGE_SHAPE, model_pathname=OPTICAL_FLOW_WEIGHTS, verbose=True)

model_graph = pwc_net.graph

with model_graph.as_default():
    mr_subnet = MaskRefineSubnet(weights_path=MASK_REFINE_WEIGHTS)
    
    mr_module = MaskRefineModule(pwc_net, mr_subnet)

## Inference (on Random Image Pairs)

The following code snippet will run prediction and evaluation for a random image from the dataset.

In [2]:
input_stack, gt_mask = next(GENERATOR)

with model_graph.as_default():
    pred_mask = mr_module.refine_mask(input_stack)
    pred_metrics = mr_module.evaluate_mask(input_stack, gt_mask)

pred_mask = pred_mask[0, ...]

print(f'\nOUTPUT RESULTS:')
for metric, val in zip(mr_module.metrics, pred_metrics):
    print(f'\t{metric}:\t{val}')
    
# define the images/masks to be displayed
prev_image = input_stack[..., :3]
curr_image = input_stack[..., 3:6]
trans_mask = input_stack[..., 6].astype(int)
pred_mask = pred_mask[..., 0].astype(int)
gt_mask = gt_mask[..., 0].astype(int)

NameError: name 'GENERATOR' is not defined

### Visualizations

In [ ]:
_blank = np.zeros(IMAGE_SHAPE).astype(int)

vis_fix_col(prev_image, curr_image,
            trans_mask, pred_mask,
            _blank, gt_mask,
            titles=['prev image', 'curr image',
                    'transformed mask', 'predicted mask',
                    '(blank)', 'gt mask'])